In [ ]:
import glob
import os

from astropy.io import fits
import numpy as np
import numpy.ma as ma
import pandas as pd

In [ ]:
working_directory = '/Users/cmartlin/Desktop/'
today = '12_09_2022'
postflash_data = pd.read_pickle('../../2022_data/Feb_2022_flc_all_stats_postflash.pkl')

In [ ]:
postflash_data=postflash_data.sort_values(by='datetime')

In [ ]:
def create_reference_file(year, working_directory, today, cadence, postflash_data):
    """This function will use input information to run the stack()
    function and create specific file names for the outputs.
    
    Parameters
    ----------
    year: int
        The year as an integer input.
    
    working_directory: str
        The path the files will be saved to. Needs trailing '\'
    
    today: str
        Format isn't important, needed to add date to filenames.
    
    cadence: int
        Number of years you want stacked. 1 = 1 year, 2 = biyearly, etc.
    
    postflash_data: pandas dataframe
        Dataframe to define the data you are using.
    
    Returns
    -------
    paths_year: str
        List of all FITS file needed to stack.
    
    outfile_year: str
        Filename for the outfile specified by year.
    
    error_outfile_year: str
        Filename for the error outfile specified by year.
    
    """
    fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)]
    shutters = ('A', 'B')
    for shutter in shutters:
        fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False) & (postflash_data['shutter'] == '{}'.format(shutter)) & (postflash_data['flash_cur'] == 'MED') & (postflash_data['flash_dur'] == 100.0)]
        if cadence == 1:
            if year == 2012:
                fullframe_pf_year = fullframe_pf[(fullframe_pf['datetime'] > '{}-01-01 00:00:00'.format(str(year))) & (fullframe_pf['datetime'] < '{}-11-14 00:00:00'.format(str(year+1)))]
            else:
                fullframe_pf_year = fullframe_pf[(fullframe_pf['datetime'] > '{}-01-01 00:00:00'.format(str(year))) & (fullframe_pf['datetime'] < '{}-01-01 00:00:00'.format(str(year+1)))]
            paths_year = fullframe_pf_year.path.tolist()
            print(len(paths_year))
            outfile_year = '{}{}_fullframe_{}_flc_stack_{}.fits'.format(working_directory,str(year), shutter, today)
            error_outfile_year = '{}{}_fullframe_{}_flc_error_stack_{}.fits'.format(working_directory,str(year), shutter, today)
            print(outfile_year)
        else:
            fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False) & (postflash_data['shutter'] == '{}'.format(shutter)) & (postflash_data['flash_cur'] == 'MED') & (postflash_data['flash_dur'] == 100.0)]
            fullframe_pf_year = fullframe_pf[(fullframe_pf['datetime'] > '{}-01-01 00:00:00'.format(str(year))) & (fullframe_pf['datetime'] < '{}-01-01 00:00:00'.format(str(year+cadence)))]
            paths_year = fullframe_pf_year.path.tolist()
            print(len(paths_year))
            outfile_year = '{}{}_cadence{}_fullframe_{}_flc_stack_{}.fits'.format(working_directory,str(year), str(cadence), shutter, today)
            error_outfile_year = '{}{}_cadence{}_fullframe_{}_flc_error_stack_{}.fits'.format(working_directory,str(year), str(cadence), shutter, today)
            print(outfile_year)

    return paths_year, outfile_year, error_outfile_year, fullframe_pf_year

In [ ]:
postflash_data=postflash_data.sort_values(by='datetime')
fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)]

start_year = '2009'
end_year = '2016'

shutters = ('A', 'B')
for shutter in shutters:
    fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False) & (postflash_data['shutter'] == '{}'.format(shutter)) & (postflash_data['flash_cur'] == 'MED') & (postflash_data['flash_dur'] == 100.0)]
    fullframe_pf_year = fullframe_pf[(fullframe_pf['datetime'] > '{}-01-01 00:00:00'.format(start_year)) & (fullframe_pf['datetime'] < '{}-01-01 00:00:00'.format(end_year))]
    paths_year = fullframe_pf_year.path.tolist()
    print(len(paths_year))
    outfile_year = '{}testing_pipeline_{}_shutter{}.fits'.format(working_directory, today, shutter)
    error_outfile_year = '{}testing_pipeline_error_stack_{}_shutter{}.fits'.format(working_directory, today, shutter)
    print(outfile_year)

In [ ]:
def stack(list_of_files, outfile, error_file):
    """This function will stack a set of FITS images and create a masked
    median file along with an error file.
    
    Parameters
    ----------
    list_of_files: str
        This is the list you create from the path column of the
        subset of the pandas database.

    outfile: str
        This is the path and filename you want for your outfile.

    error_file: str
    This is the path and filename you want for your
        outfile of the calcuated error.
    
    """
    # Gets the file size
    hdr = fits.getheader(list_of_files[0], 1)
    nx = hdr['NAXIS1']
    ny = hdr['NAXIS2']
    nf = len(list_of_files)
    
    # Setting up the empty data, rms, and error arrays
    data_array_1 = np.empty((nf, ny, nx), dtype=float)
    data_array_2 = np.empty((nf, ny, nx), dtype=float)
    set_data=fits.getdata(list_of_files[0], 1)
    rms_1 = np.zeros(len(list_of_files), dtype=float)
    rms_2 = np.zeros(len(list_of_files), dtype=float)
    error_array_1 = np.empty((nf, ny, nx), dtype=float)
    error_array_2 = np.empty((nf, ny, nx), dtype=float)
    total_error_1 = np.zeros_like(set_data, dtype=float)
    total_error_2 = np.zeros_like(set_data, dtype=float)
    
    for i , f in enumerate(list_of_files):
        #Read in the data and the DQs from the FITS for both extensions
        data_1 = fits.getdata(f, 1)
        data_2 = fits.getdata(f, 4)
        error_1 = fits.getdata(f, 2)
        error_2 = fits.getdata(f, 5)
        DQ_1 = fits.getdata(f, 3)
        DQ_2 = fits.getdata(f, 6)
        
        #Set the mask to a boolean array of the same size and shape as the data array
        mask_1 = np.zeros_like(data_1, dtype=bool)
        mask_2 = np.zeros_like(data_2, dtype=bool)
        
        #DQ = true because in the masking step a 1 will be masked and 0 allowed,
        #so the logic in this step must be reversed.
        #Create a mask for the data by setting any pixel with the flag value to True.
        mask_1[DQ_1>=2**13] = True
        mask_2[DQ_2>=2**13] = True
        error_1[(0<DQ_1 & (DQ_1<2**13))] = 0.00001
        error_2[(0<DQ_2& (DQ_2<2**13))] = 0.00001
        error_1_sq = error_1**2
        error_2_sq = error_2**2
        
        #Mask the data
        masked_data_1 = ma.array(data=data_1, mask=mask_1)
        masked_data_2 = ma.array(data=data_2, mask=mask_2)
        
        #Resets the data to an array for stacking
        data_array_1[i, :, :] = masked_data_1
        rms_1[i] = masked_data_1.std()
        data_array_2[i, :, :] = masked_data_2
        rms_2[i] = masked_data_2.std()
        total_error_1 = total_error_1+(error_1_sq)
        total_error_2 = total_error_2+(error_2_sq)
        
    sr_total_error_1 = np.sqrt(total_error_1)
    sr_total_error_2 = np.sqrt(total_error_2)
    fin_error_1 = (sr_total_error_1/(float(len(list_of_files))))
    fin_error_2 = (sr_total_error_2/(float(len(list_of_files))))
    
    #Create the median image
    image_median_1 = np.median(data_array_1, axis=0)
    image_median_2 = np.median(data_array_2, axis=0)
    new_hdul = fits.HDUList()
    new_hdul.append(fits.ImageHDU(image_median_1))
    new_hdul.append(fits.ImageHDU(image_median_2))
    new_hdul.writeto(outfile, overwrite=True)
    
    #Error
    new_hdul = fits.HDUList()
    new_hdul.append(fits.ImageHDU(fin_error_1))
    new_hdul.append(fits.ImageHDU(fin_error_2))
    new_hdul.writeto(error_file,overwrite=True)

In [ ]:
stack(paths_year, outfile_year, error_outfile_year)